# The Battle of Neighborhoods: John Data is looking for the best neighborhood to live in Vancouver

## 1. Problem Description

John Data is going to start a Master program at the University of British Columbia at Vancouver and he’s looking for an apartment to rent.

He tried to look for the nearest apartment to the school, but they are out of his budget, so he wants to look for other options. John already checked the price range of the apartments and the best neighborhoods are Kitsilano, Fairview, West Point Grey, and Arbutus-Ridge the only thing that keeps him from deciding is the venues near each neighborhood in which he could work part-time.

Each neighborhood will be analyzed to select the one with the most venues where he could work at and create a list of venues


## 2. Data Description

### 2.1. City of Vancouver Open Data Portal

The data of the neighborhoods and their location will be taken directly from this portal. The CSV file will be used to get the latitude and longitude of the specified neighborhoods

Data Link: https://opendata.vancouver.ca/explore/dataset/local-area-boundary/table/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6ImxvY2FsLWFyZWEtYm91bmRhcnkiLCJvcHRpb25zIjp7ImxvY2F0aW9uIjoiMTIsNDkuMjQ3NCwtMTIzLjEyNDAyIn19LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjMDI3OUIxIn1dLCJ4QXhpcyI6Im5hbWUiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D&location=12,49.2474,-123.12402

CSV File Location: https://opendata.vancouver.ca/explore/dataset/local-area-boundary/download/?format=csv&timezone=America/Denver&lang=en&use_labels_for_header=true&csv_separator=%3B

The information obtained from this DB is:

1.	MapID: The identifier of the neighborhood
2.	Name: Official name of the Neighborhood
3.	Geom: Spatial representation of Local Area
4.	geo_point_2d: Latitude and Longitude of the neighborhood


### b) Foursquare

The data about the venues located in each of the neighborhoods selected by John is needed. Foursquare will be used in order to get the locational information. This data provider has information about the venues (such as names, locations, menus and more) per location and will be helpful to get the data that we need.

The data retrieved from Foursquare contained information of venues within a radius of 500 meters of the longitude and latitude of each neighborhood. The information obtained per venue as follows:

1.	Neighborhood
2.	Neighborhood Latitude
3.	Neighborhood Longitude
4.	Venue
5.	Name of the venue e.g. the name of a store or restaurant
6.	Venue Latitude
7.	Venue Longitude
8.	Venue Category


## 3. Methodology

In [1]:
import pandas as pd
import requests as req
import io
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium # plotting library

     |████████████████████████████████| 79 kB 6.3 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=5f83e21a9421f55be5ab6c3d8a52ef7468d8155b2eca00ba948a859cd75b10c5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [2]:
url = "https://opendata.vancouver.ca/explore/dataset/local-area-boundary/download/?format=csv&timezone=America/Denver&lang=en&use_labels_for_header=true&csv_separator=%3B"

In [3]:
s = req.get(url).content

In [4]:
c = pd.read_csv(io.StringIO(s.decode(encoding='cp1252')),error_bad_lines=False,sep=';')

In [5]:
c

,MAPID,Name,Geom,geo_point_2d
0,AR,Arbutus-Ridge,"{""type"": ""Polygon"", ""coordinates"": [[[-123.152...","49.2468049108,-123.161669238"
1,CBD,Downtown,"{""type"": ""Polygon"", ""coordinates"": [[[-123.112...","49.2807470711,-123.116567008"
2,FAIR,Fairview,"{""type"": ""Polygon"", ""coordinates"": [[[-123.145...","49.2645404871,-123.131048865"
3,GW,Grandview-Woodland,"{""type"": ""Polygon"", ""coordinates"": [[[-123.077...","49.2764396102,-123.066728221"
4,HS,Hastings-Sunrise,"{""type"": ""Polygon"", ""coordinates"": [[[-123.056...","49.277934053,-123.040269923"
5,MARP,Marpole,"{""type"": ""Polygon"", ""coordinates"": [[[-123.106...","49.2102074666,-123.12838166"
6,RP,Riley Park,"{""type"": ""Polygon"", ""coordinates"": [[[-123.105...","49.2447664796,-123.103146806"
7,SHAU,Shaughnessy,"{""type"": ""Polygon"", ""coordinates"": [[[-123.155...","49.2456809771,-123.139760167"
8,STR,Strathcona,"{""type"": ""Polygon"", ""coordinates"": [[[-123.099...","49.2782195786,-123.088235281"
9,WE,West End,"{""type"": ""Polygon"", ""coordinates"": [[[-123.137...","49.2850111894,-123.135437653"


In [6]:
c[['lat','lon']] = c.geo_point_2d.str.split(",",expand=True)

In [7]:
c.drop(columns=['geo_point_2d','Geom'])

,MAPID,Name,lat,lon
0,AR,Arbutus-Ridge,49.2468049108,-123.161669238
1,CBD,Downtown,49.2807470711,-123.116567008
2,FAIR,Fairview,49.2645404871,-123.131048865
3,GW,Grandview-Woodland,49.2764396102,-123.066728221
4,HS,Hastings-Sunrise,49.277934053,-123.040269923
5,MARP,Marpole,49.2102074666,-123.12838166
6,RP,Riley Park,49.2447664796,-123.103146806
7,SHAU,Shaughnessy,49.2456809771,-123.139760167
8,STR,Strathcona,49.2782195786,-123.088235281
9,WE,West End,49.2850111894,-123.135437653


### - Now we'll specify the neighborhood we want to analyze -

In [8]:
#Kitsilano
Kitlat = (c[c['Name'].str.contains("Kitsilano")].reset_index(drop=True)).loc[0]["lat"]
Kitlon = (c[c['Name'].str.contains("Kitsilano")].reset_index(drop=True)).loc[0]["lon"]


#Fairview
Fairlat = (c[c['Name'].str.contains("Fairview")].reset_index(drop=True)).loc[0]["lat"]
Fairlon = (c[c['Name'].str.contains("Fairview")].reset_index(drop=True)).loc[0]["lon"]


#West Point Grey
Westlat = (c[c['Name'].str.contains("West Point Grey")].reset_index(drop=True)).loc[0]["lat"]
Westlon = (c[c['Name'].str.contains("West Point Grey")].reset_index(drop=True)).loc[0]["lon"]


#Arbutus-Ridge
Arblat = (c[c['Name'].str.contains("Arbutus-Ridge")].reset_index(drop=True)).loc[0]["lat"]
Arblon = (c[c['Name'].str.contains("Arbutus-Ridge")].reset_index(drop=True)).loc[0]["lon"]

print("Kitsilano:","(",Kitlat , Kitlon,")")
print("Fairview:","(",Fairlat , Fairlon,")")
print("West Point Grey:","(",Westlat , Westlon,")")
print("Arbutus-Ridge:","(",Arblat , Arblon,")")

Kitsilano: ( 49.2675398494 -123.16329474 )
Fairview: ( 49.2645404871 -123.131048865 )
West Point Grey: ( 49.2684012111 -123.203467483 )
Arbutus-Ridge: ( 49.2468049108 -123.161669238 )


### - We'll explore the venues near each neighborhood (500 m) using Foursquare - 

In [9]:
CLIENT_ID = 'PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB' 
CLIENT_SECRET = 'ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3' 
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB
CLIENT_SECRET:ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3


In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
rad = 500

#Kitsilano
Kiturl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Kitlat, 
    Kitlon, 
    rad, 
    LIMIT)

#Fairview
Fairurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Fairlat, 
    Fairlon, 
    rad, 
    LIMIT)


#West Point Grey
Westurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Westlat, 
    Westlon, 
    rad, 
    LIMIT)


#Arbutus-Ridge
Arburl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Arblat, 
    Arblon, 
    rad, 
    LIMIT)

print(Kiturl)
print(Fairurl)
print(Westurl)
print(Arburl)

https://api.foursquare.com/v2/venues/explore?&client_id=PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB&client_secret=ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3&v=20180604&ll=49.2675398494,-123.16329474&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB&client_secret=ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3&v=20180604&ll=49.2645404871,-123.131048865&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB&client_secret=ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3&v=20180604&ll=49.2684012111,-123.203467483&radius=500&limit=100
https://api.foursquare.com/v2/venues/explore?&client_id=PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB&client_secret=ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3&v=20180604&ll=49.2468049108,-123.161669238&radius=500&limit=100


In [12]:
Kit_ven = (req.get(Kiturl).json())['response']['groups'][0]['items']
Fair_ven = (req.get(Fairurl).json())['response']['groups'][0]['items']
West_ven = (req.get(Westurl).json())['response']['groups'][0]['items']
Arb_ven = (req.get(Arburl).json())['response']['groups'][0]['items']


Kitnearby_ven = json_normalize(Kit_ven)
Fairnearby_ven = json_normalize(Fair_ven)
Westnearby_ven = json_normalize(West_ven)
Arbnearby_ven = json_normalize(Arb_ven)

Kitnearby_ven

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.venuePage.id,venue.location.neighborhood
0,e-0-57a58607498e40faba10a1f6-0,0,"[{'summary': 'This spot is popular', 'type': '...",57a58607498e40faba10a1f6,The Only Cafe,2678 W 4th Ave,49.268197,-123.165536,"[{'label': 'display', 'lat': 49.26819659535481...",178,...,Vancouver,BC,Canada,"[2678 W 4th Ave, Vancouver BC V6K 1P7, Canada]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN
1,e-0-5810f05238faa94dbabd4542-1,0,"[{'summary': 'This spot is popular', 'type': '...",5810f05238faa94dbabd4542,Cafe Lokal,2610 W 4th Ave,49.268174,-123.164710,"[{'label': 'display', 'lat': 49.26817419279799...",124,...,Vancouver,BC,Canada,"[2610 W 4th Ave (Trafalgar), Vancouver BC, Can...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],Trafalgar,NaN,NaN
2,e-0-4fa03d00e4b0829ddef429d9-2,0,"[{'summary': 'This spot is popular', 'type': '...",4fa03d00e4b0829ddef429d9,Guanaco Salvadoran Cuisine food truck,Down town Vancouver,49.268251,-123.161749,"[{'label': 'display', 'lat': 49.26825059811553...",137,...,Vancouver,BC,Canada,[Down town Vancouver (Burrard St. & W. Pender ...,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",0,[],Burrard St. & W. Pender St,43353999,NaN
3,e-0-4aaa6f4ff964a520ef5520e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4aaa6f4ff964a520ef5520e3,Terra Breads,2380 W 4th Ave,49.268139,-123.159275,"[{'label': 'display', 'lat': 49.26813882669247...",299,...,Vancouver,BC,Canada,"[2380 W 4th Ave (at Balsam St), Vancouver BC V...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],at Balsam St,NaN,NaN
4,e-0-5780683c498e51523ab4851b-4,0,"[{'summary': 'This spot is popular', 'type': '...",5780683c498e51523ab4851b,Raisu,2340 West 4th Ave.,49.268244,-123.158430,"[{'label': 'display', 'lat': 49.2682441750864,...",361,...,Vancouver,BC,Canada,"[2340 West 4th Ave., Vancouver BC V6K 1P1, Can...","[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",0,[],NaN,NaN,NaN
5,e-0-505d3c21e4b06d1d2af233e4-5,0,"[{'summary': 'This spot is popular', 'type': '...",505d3c21e4b06d1d2af233e4,Dark Table,"2611, West 4th Avenue",49.268322,-123.164663,"[{'label': 'display', 'lat': 49.26832206854146...",132,...,Vancouver,BC,Canada,"[2611, West 4th Avenue (at Trafalger St), Vanc...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],at Trafalger St,NaN,NaN
6,e-0-4afde409f964a520cf2b22e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4afde409f964a520cf2b22e3,Thomas Haas Patisserie,2539 W Broadway,49.263974,-123.163260,"[{'label': 'display', 'lat': 49.26397426489346...",396,...,Vancouver,BC,Canada,"[2539 W Broadway (btwn Trafalgar & Larch), Van...","[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",0,[],btwn Trafalgar & Larch,NaN,NaN
7,e-0-4aa7d91ef964a520944d20e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4aa7d91ef964a520944d20e3,The Naam Restaurant,2724 W 4th Ave,49.268301,-123.167049,"[{'label': 'display', 'lat': 49.26830060746881...",285,...,Vancouver,BC,Canada,"[2724 W 4th Ave (at Stephens St), Vancouver BC...","[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",0,[],at Stephens St,NaN,NaN
8,e-0-5453fb6d498e366a912293d5-8,0,"[{'summary': 'This spot is popular', 'type': '...",5453fb6d498e366a912293d5,Au Comptoir,2278 W 4th Ave,49.268195,-123.157040,"[{'label': 'display', 'lat': 49.26819489559636...",460,...,Vancouver,BC,Canada,"[2278 W 4th Ave (btwn Vine & Yew), Vancouver B...","[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",0,[],btwn Vine & Yew,NaN,NaN
9,e-0-4aa9a9c7f964a520bf5420e3-9,0,"[{'summary': 'This spot is popular', 'type': '...",4aa9a9c7f964a520bf5420e3,Nat's New York Pizzeria,2684 West Broadway,49.264026,-123.165995,"[{'label

In [13]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

#Kitsilano
Kitnearby_ven = Kitnearby_ven.loc[:, filtered_columns]
Kitnearby_ven['venue.categories'] = Kitnearby_ven.apply(get_category_type, axis=1)
Kitnearby_ven.columns = [col.split(".")[-1] for col in Kitnearby_ven.columns]

#Fairview
Fairnearby_ven = Fairnearby_ven.loc[:, filtered_columns]
Fairnearby_ven['venue.categories'] = Fairnearby_ven.apply(get_category_type, axis=1)
Fairnearby_ven.columns = [col.split(".")[-1] for col in Fairnearby_ven.columns]

#West Point Grey
Westnearby_ven = Westnearby_ven.loc[:, filtered_columns]
Westnearby_ven['venue.categories'] = Westnearby_ven.apply(get_category_type, axis=1)
Westnearby_ven.columns = [col.split(".")[-1] for col in Westnearby_ven.columns]

#Arbutus-Ridge
Arbnearby_ven = Arbnearby_ven.loc[:, filtered_columns]
Arbnearby_ven['venue.categories'] = Arbnearby_ven.apply(get_category_type, axis=1)
Arbnearby_ven.columns = [col.split(".")[-1] for col in Arbnearby_ven.columns]



## 4. Results

### 4.1. Neighborhood with most venues

In [14]:
print("Kitsilano has", Kitnearby_ven['name'].count(),"venues")
print("Fairview has", Fairnearby_ven['name'].count(),"venues")
print("West Point Grey has", Westnearby_ven['name'].count(),"venues")
print("Arbutus-Ridge has", Arbnearby_ven['name'].count(),"venues")


Kitsilano has 49 venues
Fairview has 26 venues
West Point Grey has 6 venues
Arbutus-Ridge has 2 venues


### 4.2. List of venues

In [15]:
Kitnearby_ven

,name,categories,lat,lng
0,The Only Cafe,Café,49.268197,-123.165536
1,Cafe Lokal,Coffee Shop,49.268174,-123.164710
2,Guanaco Salvadoran Cuisine food truck,Food Truck,49.268251,-123.161749
3,Terra Breads,Bakery,49.268139,-123.159275
4,Raisu,Japanese Restaurant,49.268244,-123.158430
5,Dark Table,Restaurant,49.268322,-123.164663
6,Thomas Haas Patisserie,Dessert Shop,49.263974,-123.163260
7,The Naam Restaurant,Vegetarian / Vegan Restaurant,49.268301,-123.167049
8,Au Comptoir,French Restaurant,49.268195,-123.157040
9,Nat's New York Pizzeria,Pizza Place,49.264026,-123.165995


## 5. Results

- The results on this analysis are based only on the venues near each neighborhood, they might or might not be hiring, but with more venues the probability of finding a part-time job increases
- A further investigation about the best place to rent was attempted but since an open database with the price and location of apartments in rent couldn’t be found, only the 4 specified neighborhoods were used. 


## 6. Conclusion

- Pandas was a great help in order to retrieve the information of the Vancouver neighborhoods, since there were some issues with the API. 
- Data analysis techniques used in this project were of great help and will be helpful in the solution of future business problems. 
- Thanks to this project I was able to pinpoint the areas that I still lack in and will keep practicing to improve.
- Although the problem in this project was created just for the sake of this assignment, investigating about the university and the area made me consider the master degree in Vancouver, and if I ever decide to go there, I will most likely improve this investigation to help me find a place to live in. 
